## Импорты

In [1]:
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms

!pip install pytorch-lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import F1Score, AUROC

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 41.5 MB/s eta 0:00:00


## DataModule для FashionMNIST

In [2]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):

        datasets.FashionMNIST(self.data_dir, train=True, download=True)
        datasets.FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Split dataset
        if stage == 'fit' or stage is None:
            full_train_dataset = datasets.FashionMNIST(self.data_dir, train=True, transform=self.transform)
            self.train_dataset, self.val_dataset = random_split(full_train_dataset, [50000, 10000])
        if stage == 'test' or stage is None:
            self.test_dataset = datasets.FashionMNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

## Реализация модели FashionMNIST с использованием PyTorch Lightning

In [3]:
class FashionMNISTModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        self.criterion = nn.CrossEntropyLoss()


        self.f1_score = F1Score(num_classes=10, task='multiclass', average='macro')
        self.auroc = AUROC(num_classes=10, task='multiclass')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)


        self.f1_score.update(preds, y)
        self.auroc.update(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):

        f1 = self.f1_score.compute()
        auroc = self.auroc.compute()
        self.log('val_f1', f1, prog_bar=True)
        self.log('val_auroc', auroc, prog_bar=True)
        self.f1_score.reset()
        self.auroc.reset()
        avg_loss = self.trainer.callback_metrics.get('val_loss', None)
        if avg_loss is not None:
            self.log('val_loss', avg_loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)


        self.f1_score.update(preds, y)
        self.auroc.update(y_hat, y)
        self.log('test_loss', loss, prog_bar=True)
        return loss

    def on_test_epoch_end(self):

        f1 = self.f1_score.compute()
        auroc = self.auroc.compute()
        self.log('test_f1', f1)
        self.log('test_auroc', auroc)
        self.f1_score.reset()
        self.auroc.reset()
        avg_loss = self.trainer.callback_metrics.get('test_loss', None)
        if avg_loss is not None:
            self.log('test_loss', avg_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        return [optimizer], [scheduler]

## Точка запуска


In [5]:
if __name__ == '__main__':

    data_module = FashionMNISTDataModule(batch_size=64)

    model = FashionMNISTModel(learning_rate=1e-3)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')
    logger = TensorBoardLogger('tb_logs', name='fashion_mnist')

    trainer = pl.Trainer(
        max_epochs=10,
        accelerator='auto',
        callbacks=[early_stopping],
        logger=logger
    )

    trainer.fit(model, data_module)
    trainer.test(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


100%|██████████| 26.4M/26.4M [00:01<00:00, 17.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 268kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_auroc         │    0.9909062385559082     │
│          test_f1          │    0.8800898790359497     │
│         test_loss         │     0.339213490486145     │
└───────────────────────────┴───────────────────────────┘

**Общие характеристики модели:**

Модель имеет 109K обучаемых параметров
Все параметры модели являются обучаемыми, нет фиксированных весов (0 не обучаемых параметров)

**Эффективность модели на тестовых данных:**

- AUROC (Area Under ROC Curve): 0.9909
- F1-метрика: 0.8800
- Тестовая потеря (test_loss): 0.3392

**Процесс обучения:**

Обучение завершилось после 9 эпох, достигнув 100% выполнения, что может свидетельствовать об использовании ранней остановки для предотвращения переобучения.